In [1]:
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
from datetime import datetime, date
import json
from xgrads import open_CtlDataset
import geopandas as gpd
import requests
import matplotlib.pyplot as plt
from geocube.api.core import make_geocube
%matplotlib inline

# In QGIS

Once you have the huc4 geojson, the streamgage station shapefiles and the filled dem downloaded - you need to do some work in QGIS to determine the upslope area of each stream gage. 

A detailed description of the workflow can be found here: https://docs.google.com/document/d/1Urdpp41tCZp0ZptHDdRD2HQ3ZznolhWA0Zaa_szcgfc/edit?usp=sharing
Here is the overview:
1. Load DEM, huc4 shapefile, and gage shapefile
2. Reproject huc4 shape to match DEM
2. Clip dem to HUC4
3. Fill Sinks tool
4. Strahler Order tool
5. Using station locations, use Upslope Area tool to save out .tif files with upslope area for each gage. 
6. Rasterize the huc4 shapefile to use for masking later. 

In [2]:
# Choose a domain
domain = 'YUKO'

# Path to the SnowModel folder
gagepath = '/nfs/attic/dfh/2020_NPRB/data/stream_gage/'+domain+'/'

#path to NPRB domains
domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
domains = domains_resp.json()

In [3]:
list_of_files = sorted( filter( os.path.isfile,
                        glob.glob(gagepath + '*.tif') ) )

In [4]:
list_of_files

['/nfs/attic/dfh/2020_NPRB/data/stream_gage/YUKO/15348000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/YUKO/15356000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/YUKO/15446000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/YUKO/15452300.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/YUKO/15453500.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/YUKO/15476000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/YUKO/15484000.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/YUKO/15515500.tif',
 '/nfs/attic/dfh/2020_NPRB/data/stream_gage/YUKO/15565447.tif']

In [5]:
for i in range(len(list_of_files)):

    file = list_of_files[i]
    stncode=file[-12:-4]
    data = rxr.open_rasterio(file)
    da = data.sel(band=1)
    if i ==0:
        ds = da.to_dataset(name=stncode)
    else:
        ds[stncode]=da
ds

<xarray.Dataset>
Dimensions:      (x: 2052, y: 1336)
Coordinates:
    band         int64 1
  * y            (y) float64 2.302e+06 2.300e+06 2.3e+06 ... 9.675e+05 9.665e+05
  * x            (x) float64 -6.215e+05 -6.205e+05 ... 1.428e+06 1.43e+06
    spatial_ref  int64 0
Data variables:
    15348000     (y, x) float32 ...
    15356000     (y, x) float32 ...
    15446000     (y, x) float32 ...
    15452300     (y, x) float32 ...
    15453500     (y, x) float32 ...
    15476000     (y, x) float32 ...
    15484000     (y, x) float32 ...
    15515500     (y, x) float32 ...
    15565447     (y, x) float32 ...

In [6]:
ds.to_netcdf(gagepath+'upslope.nc')